In [1]:
import os,sys,math,numpy
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt

# you must have openvisus installed
# python -m pip install --upgrade OpenVisus
from OpenVisus import *

# LoadImage (example: img=LoadImage("https://github.com/sci-visus/OpenVisus/raw/master/datasets/cat/gray.png"))
def LoadImage(filename):
	return numpy.asarray(PIL.Image.open(urlopen(filename)  if filename.startswith('http') else filename))

Starting OpenVisus c:\python38\lib\site-packages\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


In [2]:
db=LoadDataset(r'D:\GoogleSci\visus_dataset\TALASS\ACAM\battery.idx')
logic_box=db.getLogicBox()
dims=logic_box[1]
ranges=[(logic_box[0][I],logic_box[1][I]) for I in range(3)]
print("logic_box",logic_box)
print("ranges",ranges)
print("dims",dims)

fields=[db.getField(name) for name in db.getFields()]
for field in fields:
    dtype=field.dtype
    GB=int(dims[0]*dims[1]*dims[2]*dtype.getByteSize()/(1024*1024*1024))
    print("Field",field.name, dtype.toString(), "{}GB".format(GB))

logic_box ([0, 0, 0], [2160, 2160, 2560])
ranges [(0, 2160), (0, 2160), (0, 2560)]
dims [2160, 2160, 2560]
Field xray float32 44GB
Field neutron float32 44GB


In [3]:
# IMPORTANT: FOR VALERIO run this
# c:\Python38\python.exe -m pip install ipywidgets
# c:\Python38\Scripts\jupyter.exe nbextension enable --py widgetsnbextension

%matplotlib notebook
import ipywidgets

MAX_PIXELS=1024*1024

def ExtractSlice(axis ,offset, num_refinements): 

    x,y,z=(
        ranges[0] if axis!=0 else [offset,offset+1],
        ranges[1] if axis!=1 else [offset,offset+1],
        ranges[2] if axis!=2 else [offset,offset+1])

    # guess quality
    quality=0
    tot_pixels=(x[1]-x[0]) * (y[1]-y[0]) * (z[1]-z[0])
    while tot_pixels >= MAX_PIXELS:
        quality-=1
        tot_pixels=int(tot_pixels/2) 

    print("Extracting slice","quality",quality,"num_refinements",num_refinements,"x",x,"y",y,"z",z)   
    
    fig = plt.figure(figsize=(6, 6),frameon=False)
    fig.tight_layout()
   
    sub = fig.add_subplot(1,1,1) 
    sub.axis('off')    
    
    while True:
        try:
            cont=0
            for data in db.read(x=x,y=y,z=z,num_refinements=num_refinements,quality=quality):
                # print("cont",cont,"{} {} quality({})".format(data.shape,data.dtype,quality));cont+=1
                if len(data.shape)==3 and data.shape[0]==1: data=data[0,:,:]
                if len(data.shape)==3 and data.shape[1]==1: data=data[:,0,:]   
                if len(data.shape)==3 and data.shape[2]==1: data=data[:,:,0] 
                sub.set_title("{} {} quality({})".format(data.shape,data.dtype,quality))
                sub.imshow(data)
                fig.canvas.draw()
            break
        except:
            #it can fail because the quality is too low
            print("increasing quality")
            quality+=1

def OnAxisChange(*args):
    axis=axis_slider.value
    offset_slider.max=dims[axis]-1
    offset_slider.value=int(dims[axis]/2)      

axis_slider=ipywidgets.IntSlider(min=0, max=2, step=1, value=0)
offset_slider=ipywidgets.IntSlider(min=0, max=dims[0]-1, step=1, value=int(dims[0]/2))
num_refinements_slider=ipywidgets.IntSlider(min=1, max=6, step=1, value=3)

axis_slider.observe(OnAxisChange, 'value')

# replace with 'interact' if you don't want to press any button
ipywidgets.interact_manual(
    ExtractSlice, 
    axis=axis_slider, 
    offset= offset_slider, 
    num_refinements=num_refinements_slider)

interactive(children=(IntSlider(value=0, description='axis', max=2), IntSlider(value=1080, description='offset…

<function __main__.ExtractSlice(axis, offset, num_refinements)>